In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
## Keypoints using MP Holistic ##

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model -> make out detections
mp_drawing = mp.solutions.drawing_utils # drawing utilities -> draw them

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion BGR to RGB
    image.flags.writeable = False # Image no longer writeable
    results = model.process(image) # Make prediction (image is the frame)
    image.flags.writeable = True   # Image is now writable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conversion back
    return image, results

In [ ]:
def draw_landmarks(image, results):
    #connection maps
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) #connection for face
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [ ]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1 ),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1 ))

    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(thickness=2, circle_radius=2))

    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(thickness=2, circle_radius=2))

In [ ]:
import cv2
#make sure we can access our webcam

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        #Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image) #img from webcam

        # Breaking gracefully by qlicking q
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
#draw_landmarks(frame, results)
#plt.imshow(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

In [ ]:
## Extract Keypoint Values ##

In [ ]:
pose = [] # placeholder array
for res in results.pose_landmarks.landmark: #extract landmarks
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
test

In [ ]:
def extract_keypoints(results):
    # Note: flatten for making it into one big array
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    return np.concatenate([pose, face, lh, rh])

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
np.save('0', result_test)

In [ ]:
np.load('0.npy')

In [ ]:
## Setup Folders for Collection ##

In [ ]:
# Var that holds path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') #were we will store the data

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# 30 videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
seq_length = 30

In [ ]:
# Folders:
# Hello
## 0
## 1 .....
## 29
# Thanks
# I Love you

In [ ]:
for action in actions:
    for seq in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(seq)))
        except:
            pass

In [ ]:
## Collect Keypoint Values For Training and Testing ##

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # NEW loop through actions
    for action in actions:
        # loop through sequences aka videos
        for seq in range(no_sequences):
            #loop through video length aka seq length
            for frame_num in range(seq_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Take a break between each video - NEW Wait logic
                if frame_num == 0:
                    cv2.putText(image, 'Starting collection', (120,200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video number {}'.format(action, seq), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video number {}'.format(action, seq), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)

                # NEW Export Keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(seq), str(frame_num))
                np.save(npy_path, keypoints)

                # Show to screen
                cv2.imshow('OpenCV Feed', image) #img from webcam

                # Breaking gracefully by clicking q
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
## Preprocess Data and Create Labels and Features ##

In [ ]:
from sklearn.model_selection import train_test_split #partion our data to train partition
from tensorflow.keras.utils import to_categorical

In [ ]:
# label map
label_map = {label:num for num , label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for seq in range(no_sequences):
        window = [] # All diff frames for particular seq
        for frame_num in range(seq_length):
            res = np.load(os.path.join(DATA_PATH, action, str(seq), "{}.npy".format(frame_num))) #looping through our frame
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
x = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int) #category using binary 0,1

In [ ]:
# Testing and training partition

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05) #0.05 -> 5% of our data

In [ ]:
## Build and Train LSTM Neural Network ##

In [ ]:
from tensorflow.keras.models import Sequential  # seq neural network
from tensorflow.keras.layers import LSTM, Dense  # temperal component -> action detection, Dense like layers
from tensorflow.keras.callbacks import TensorBoard # logging

In [ ]:
#log directory

log_dir = os.path.join('Logs') # web app allows u to monitor ur training
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential() # Creates an instance of the Sequential class
# Sets
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu')) # false cuz next line is Dense
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # 3 neural network units

In [ ]:
# this loss model u should use for multi cross classification model
# if binary cross model u use binary crossentropy

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# train our model
# 2000 epochs too high here btw
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
## Make Predictions ##

In [ ]:
res = model.predict(x_test)

In [ ]:
actions[np.argmax(res[0])]

In [ ]:
actions[np.argmax(y_test[0])] # performing good

In [ ]:
## Save Weights ##

In [ ]:
model.save('action.h5')

In [ ]:
model.save('actions.keras')

In [ ]:
## Evaluation Using Confusion Matrix and Accuracy ##

In [ ]:
# help us to evaluate what has been detected as true positive and true negative or false ....

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(x_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
# shape 2*2 -> we want numbers to be in top left corner or bottom right corner
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat) # if 1.0 then 100% accuracy in our test set (our set was small though)

In [ ]:
## Test in Real Life ##

In [ ]:
#1. New detection variables
sequence = [] # collect our 30 frames to predict
sentence = [] # concatenate history of detection
threshold = 0.4 # confidence matrix

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        #Draw landmarks
        draw_styled_landmarks(image, results)

        #2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.insert(0, keypoints)
        #sequence.append(keypoints)
        sequence = sequence[:30]  # last 30 frames

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

        # 3. Visualization Logic
        if res[np.argmax(res)] > threshold: # highest score result > threshold
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]: # checking if the word matches the last prediction
                    sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)]) # if no sentence

        if len(sentence) > 5:
            sentence = sentence[-5:] # grabbing last 5 values

        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (225, 225, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image) #img from webcam

        # Breaking gracefully by qlicking q
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))


        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:

                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()